In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [3]:
#sample data
dataset=["The amount of pollution is increasing day by day",
          "The concert was just great",
          "I love to see Gordon Ramsay cook",
          "Google is introducing a new technology",
          "AI robots are examples of great technology present today",
          "All of us were singing in the concert",
          "We have launched campaings to stop pollution and global warming"]

In [4]:
type(dataset)

list

In [5]:
#pre-processing
dataset=[line.lower() for line in dataset]

In [6]:
dataset

['the amount of pollution is increasing day by day',
 'the concert was just great',
 'i love to see gordon ramsay cook',
 'google is introducing a new technology',
 'ai robots are examples of great technology present today',
 'all of us were singing in the concert',
 'we have launched campaings to stop pollution and global warming']

In [11]:
vectorizer=TfidfVectorizer()
x=vectorizer.fit_transform(dataset)

In [13]:
print(x[0])#Tfidf values(0,23)-first document and the first word

  (0, 5)	0.326425447033964
  (0, 9)	0.652850894067928
  (0, 17)	0.326425447033964
  (0, 19)	0.270961154226111
  (0, 25)	0.270961154226111
  (0, 24)	0.231608612212116
  (0, 2)	0.326425447033964
  (0, 33)	0.231608612212116


In [14]:
#now we are going to de-compose the Tfid matrix into three matrices
lsa=TruncatedSVD(n_components=4,n_iter=100)#n_com number of concepts we want to find
lsa.fit(x)

TruncatedSVD(n_components=4, n_iter=100)

In [15]:
row1=lsa.components_[0]#the row 1
row1

array([0.12230514, 0.17593921, 0.11986219, 0.02391177, 0.12230514,
       0.11986219, 0.02391177, 0.33986475, 0.00740591, 0.23972439,
       0.12230514, 0.02391177, 0.09746251, 0.00740591, 0.29534387,
       0.02391177, 0.17593921, 0.11986219, 0.09746251, 0.18039817,
       0.23349403, 0.02391177, 0.00740591, 0.09746251, 0.29665909,
       0.11934473, 0.12230514, 0.00740591, 0.12230514, 0.00740591,
       0.17593921, 0.02391177, 0.18242602, 0.37555093, 0.02599636,
       0.12230514, 0.17593921, 0.02391177, 0.23349403, 0.02391177,
       0.17593921])

In [16]:
print(len(row1))

41


In [17]:
terms=vectorizer.get_feature_names_out()
terms[:10]

array(['ai', 'all', 'amount', 'and', 'are', 'by', 'campaings', 'concert',
       'cook', 'day'], dtype=object)

In [20]:
#now the main job, loop through different rows!
for i, comp in enumerate(lsa.components_):#enumerate return index and row, a list of tuples
    componentTerms=zip(terms,comp)#we use zip to combine values and terms
    #sort component terms, by concept value, lambda x (x correspinding tuples) and x[1] the values
    sortedTerms=sorted(componentTerms,key=lambda x:x[1],reverse=True)#decending order
    sortedTerms=sortedTerms[:10]#select 10 most imp. terms in a specific concept
    print("\nConcept",i,":")
    for term in sortedTerms:
      print(term)


Concept 0 :
('the', 0.3755509328990523)
('concert', 0.33986475173925246)
('of', 0.29665909350149094)
('great', 0.29534386566053733)
('day', 0.23972438835590146)
('just', 0.23349402804713218)
('was', 0.23349402804713218)
('technology', 0.18242602421743648)
('is', 0.18039816898768044)
('all', 0.17593921279666547)

Concept 1 :
('to', 0.35489414747820325)
('pollution', 0.23747906096178845)
('cook', 0.22373677054461025)
('gordon', 0.22373677054461025)
('love', 0.22373677054461025)
('ramsay', 0.22373677054461025)
('see', 0.22373677054461025)
('and', 0.20380230261581317)
('campaings', 0.20380230261581317)
('global', 0.20380230261581317)

Concept 2 :
('technology', 0.36817515941203927)
('google', 0.3182850693158717)
('introducing', 0.3182850693158717)
('new', 0.3182850693158717)
('is', 0.29459629228393935)
('are', 0.1252535675825632)
('examples', 0.1252535675825632)
('present', 0.1252535675825632)
('robots', 0.1252535675825632)
('today', 0.1252535675825632)

Concept 3 :
('day', 0.392805922897

In [21]:
#the above results are almost accurate, not perfect, but can form different concepts
#Let's now see which document is from what concept
#first we need to identify what are different keywords corresponding to each concept
#we have to store sorted terms
concept_words={}#create an empty dictionary

In [22]:
for i, comp in enumerate(lsa.components_):#enumerate return index and row, a list of tuples
    componentTerms=zip(terms,comp)#we use zip to combine values and terms
    #sort component terms, by concept value, lambda x(x corresponding tuples) and X[l] the value
    sortedTerms=sorted(componentTerms,key=lambda x:x[1],reverse=True)#decending order
    sortedTerms=sortedTerms[:10]#select 10 most imp.terms in a specific concept
    concept_words["Concept "+str(i)]=sortedTerms#all concepts mapped with list of tuples

In [23]:
concept_words
#four diff concepts and for each one top 10 words, and each tuple word and corresponding concept value

{'Concept 0': [('the', 0.3755509328990523),
  ('concert', 0.33986475173925246),
  ('of', 0.29665909350149094),
  ('great', 0.29534386566053733),
  ('day', 0.23972438835590146),
  ('just', 0.23349402804713218),
  ('was', 0.23349402804713218),
  ('technology', 0.18242602421743648),
  ('is', 0.18039816898768044),
  ('all', 0.17593921279666547)],
 'Concept 1': [('to', 0.35489414747820325),
  ('pollution', 0.23747906096178845),
  ('cook', 0.22373677054461025),
  ('gordon', 0.22373677054461025),
  ('love', 0.22373677054461025),
  ('ramsay', 0.22373677054461025),
  ('see', 0.22373677054461025),
  ('and', 0.20380230261581317),
  ('campaings', 0.20380230261581317),
  ('global', 0.20380230261581317)],
 'Concept 2': [('technology', 0.36817515941203927),
  ('google', 0.3182850693158717),
  ('introducing', 0.3182850693158717),
  ('new', 0.3182850693158717),
  ('is', 0.29459629228393935),
  ('are', 0.1252535675825632),
  ('examples', 0.1252535675825632),
  ('present', 0.1252535675825632),
  ('robots

In [28]:
#let's now loop through all concepts and for each of them, print diff sentence scores
#based on values we will konw if that sentence is in that concept or not
for key in concept_words.keys():
  sentence_scores=[]#a list to store all scores for sentences and concepts
  for sentence in dataset:
    words=nltk.word_tokenize(sentence)
    score=0
    for word in words:
      for word_with_score in concept_words[key]:
        if word==word_with_score[0]:
          score+=word_with_score[1]
    sentence_scores.append(score)
  print("\n"+key+":")
  for sentence_scores in sentence_scores:
      print(sentence_scores)


Concept 0:
1.3320569721000266
1.4777476063931065
0
0.36282419320511694
0.7744289833794648
1.188013990936461
0

Concept 1:
0.23747906096178845
0
1.4735780002012546
0
0
0
1.203780116287431

Concept 2:
0.29459629228393935
0
0
1.6176266596435935
0.9944429973248553
0
0

Concept 3:
1.852977324914459
0.1031608282381785
0
0.1587942790036925
0
0.1031608282381785
0.4083629153112205
